In [1]:
!nvidia-smi 

Thu Mar 21 11:20:13 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.39       Driver Version: 418.39       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    28W /  70W |     80MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
# pytorch 
from proxyless_nas import proxyless_cpu, proxyless_gpu, proxyless_mobile, proxyless_mobile_14,model_zoo
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.utils.data
from torchvision import transforms, datasets
from proxyless_nas.utils import AverageMeter, accuracy


In [3]:
import os,time

In [4]:
data_path = '/home/qnkhuat/data/emotion_compilation_split'

In [5]:
net = proxyless_cpu(pretrained=True) # Yes, we provide pre-trained models!

In [6]:
model_names = sorted(name for name in model_zoo.__dict__
                     if name.islower() and not name.startswith("__")
                     and callable(model_zoo.__dict__[name]))
model_names

['download_url',
 'partial',
 'proxyless_base',
 'proxyless_cpu',
 'proxyless_gpu',
 'proxyless_mobile',
 'proxyless_mobile_14']

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [8]:
arch = 'proxyless_cpu'
net = model_zoo.__dict__[arch](pretrained=True)


In [12]:
data_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(os.path.join(data_path, "train"), transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
#         transforms.Normalize(
#             mean=[0.485, 0.456, 0.406],
#             std=[0.229, 0.224, 0.225]
#         ),
    ])), batch_size=32, shuffle=True, num_workers=4, pin_memory=True, drop_last=False,
)


In [13]:

# linear scale the devices

net = torch.nn.DataParallel(net).cuda()
# net = torch.nn.DataParallel(net)
cudnn.benchmark = True
criterion = nn.CrossEntropyLoss().cuda()
# criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-2)

In [14]:
total_step = len(data_loader)
for epoch in range(3):
    for i, (images, labels) in enumerate(data_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = net(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, 3, i+1, total_step, loss.item()))


Epoch [1/3], Step [100/761], Loss: 1.5196
Epoch [1/3], Step [200/761], Loss: 1.5976
Epoch [1/3], Step [300/761], Loss: 1.5327
Epoch [1/3], Step [400/761], Loss: 1.9159
Epoch [1/3], Step [500/761], Loss: 1.6400
Epoch [1/3], Step [600/761], Loss: 1.4511
Epoch [1/3], Step [700/761], Loss: 1.1470
Epoch [2/3], Step [100/761], Loss: 1.2327
Epoch [2/3], Step [200/761], Loss: 1.2544
Epoch [2/3], Step [300/761], Loss: 0.8767
Epoch [2/3], Step [400/761], Loss: 0.7437
Epoch [2/3], Step [500/761], Loss: 1.2088
Epoch [2/3], Step [600/761], Loss: 1.0424
Epoch [2/3], Step [700/761], Loss: 0.7462
Epoch [3/3], Step [100/761], Loss: 0.9478
Epoch [3/3], Step [200/761], Loss: 0.9592
Epoch [3/3], Step [300/761], Loss: 0.6489
Epoch [3/3], Step [400/761], Loss: 1.1096
Epoch [3/3], Step [500/761], Loss: 0.8954
Epoch [3/3], Step [600/761], Loss: 0.9983
Epoch [3/3], Step [700/761], Loss: 0.7898


In [ ]:
valid_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(os.path.join(data_path, "valid"), transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])), batch_size=32, shuffle=True, num_workers=1, pin_memory=True, drop_last=False,
)



In [ ]:

# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in valid_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint